# Quantitative Value Strategy
"Value investing" means investing in the stocks that are cheapest relative to common measures of business value (like earnings or assets).

For this project, we're going to build an investing strategy that selects the 50 stocks with the best value metrics. From there, we will calculate recommended trades for an equal-weight portfolio of these 50 stocks.

## Library Imports
The first thing we need to do is import the open-source software libraries that we'll be using in this tutorial.

In [1]:
import numpy as np
import xlsxwriter 
import pandas as pd 
import requests 
from scipy import stats
import math 

## Importing Our List of Stocks & API Token
As before, we'll need to import our list of stocks and our API token before proceeding. Make sure the .csv file is still in your working directory and import it with the following command:

In [2]:
stocks = pd.read_csv("sp_500_stocks.csv")
stocks

,Ticker
0,A
1,AAL
2,AAP
3,AAPL
4,ABBV
...,...
500,YUM
501,ZBH
502,ZBRA
503,ZION


In [3]:
from secrets import IEX_CLOUD_API_TOKEN

## Making Our First API Call
It's now time to make the first version of our value screener!

We'll start by building a simple value screener that ranks securities based on a single metric (the price-to-earnings ratio).

In [4]:
symbol = 'aapl'
api_url = f'https://sandbox.iexapis.com/stable/stock/{symbol}/quote?token={IEX_CLOUD_API_TOKEN}'

data = requests.get(api_url).json()
print(data)

{'symbol': 'AAPL', 'companyName': 'Apple Inc', 'primaryExchange': 'DNCTSAMGQ T(NLAE/KOSGE LB)LA ERAS', 'calculationPrice': 'close', 'open': 134.72, 'openTime': 1679556414872, 'openSource': 'olfifica', 'close': 135.04, 'closeTime': 1635057387586, 'closeSource': 'aiifcofl', 'high': 137, 'highTime': 1634721155537, 'highSource': 'dpelm drcui5ate einy 1e', 'low': 135.3, 'lowTime': 1612770241889, 'lowSource': 'iaedctnp1udrleee  y im5', 'latestPrice': 135.28, 'latestSource': 'Close', 'latestTime': 'December 24, 2020', 'latestUpdate': 1659649799618, 'latestVolume': 56242691, 'iexRealtimePrice': None, 'iexRealtimeSize': None, 'iexLastUpdated': None, 'delayedPrice': 133.32, 'delayedPriceTime': 1642604351860, 'oddLotDelayedPrice': 137.77, 'oddLotDelayedPriceTime': 1624028084640, 'extendedPrice': 133, 'extendedChange': 0.03, 'extendedChangePercent': 0.00023, 'extendedPriceTime': 1660762163400, 'previousClose': 131.75, 'previousVolume': 92583232, 'change': 1.05, 'changePercent': 0.00808, 'volume': 

## Parsing Our API Call
This API call has the metric we need - the price-to-earnings ratio.

Here is an example of how to parse the metric from our API call:

In [5]:
price = data['latestPrice']
pe_ratio = data['peRatio']
print(price, pe_ratio)

135.28 41.08


## Executing A Batch API Call & Building Our DataFrame

Just like in our first project, it's now time to execute several batch API calls and add the information we need to our DataFrame.

We'll start by running the following code cell, which contains some code we already built last time that we can re-use for this project. More specifically, it contains a function called chunks that we can use to divide our list of securities into groups of 100.

In [6]:
# Function sourced from 
# https://stackoverflow.com/questions/312443/how-do-you-split-a-list-into-evenly-sized-chunks
def chunks(lst, n):
    """Yield successive n-sized chunks from lst."""
    for i in range(0, len(lst), n):
        yield lst[i:i + n]   
        
symbol_groups = list(chunks(stocks['Ticker'], 100))
symbol_strings = []
for i in range(0, len(symbol_groups)):
    symbol_strings.append(','.join(symbol_groups[i]))
    print(symbol_strings[i])

my_columns = ['Ticker', 'Price', 'Price-to-Earnings Ratio', 'Number of Shares to Buy']

A,AAL,AAP,AAPL,ABBV,ABC,ABMD,ABT,ACN,ADBE,ADI,ADM,ADP,ADSK,AEE,AEP,AES,AFL,AIG,AIV,AIZ,AJG,AKAM,ALB,ALGN,ALK,ALL,ALLE,ALXN,AMAT,AMCR,AMD,AME,AMGN,AMP,AMT,AMZN,ANET,ANSS,ANTM,AON,AOS,APA,APD,APH,APTV,ARE,ATO,ATVI,AVB,AVGO,AVY,AWK,AXP,AZO,BA,BAC,BAX,BBY,BDX,BEN,BF.B,BIIB,BIO,BK,BKNG,BKR,BLK,BLL,BMY,BR,BRK.B,BSX,BWA,BXP,C,CAG,CAH,CARR,CAT,CB,CBOE,CBRE,CCI,CCL,CDNS,CDW,CE,CERN,CF,CFG,CHD,CHRW,CHTR,CI,CINF,CL,CLX,CMA,CMCSA
CME,CMG,CMI,CMS,CNC,CNP,COF,COG,COO,COP,COST,COTY,CPB,CPRT,CRM,CSCO,CSX,CTAS,CTL,CTSH,CTVA,CTXS,CVS,CVX,CXO,D,DAL,DD,DE,DFS,DG,DGX,DHI,DHR,DIS,DISCA,DISCK,DISH,DLR,DLTR,DOV,DOW,DPZ,DRE,DRI,DTE,DUK,DVA,DVN,DXC,DXCM,EA,EBAY,ECL,ED,EFX,EIX,EL,EMN,EMR,EOG,EQIX,EQR,ES,ESS,ETFC,ETN,ETR,EVRG,EW,EXC,EXPD,EXPE,EXR,F,FANG,FAST,FB,FBHS,FCX,FDX,FE,FFIV,FIS,FISV,FITB,FLIR,FLS,FLT,FMC,FOX,FOXA,FRC,FRT,FTI,FTNT,FTV,GD,GE,GILD
GIS,GL,GLW,GM,GOOG,GOOGL,GPC,GPN,GPS,GRMN,GS,GWW,HAL,HAS,HBAN,HBI,HCA,HD,HES,HFC,HIG,HII,HLT,HOLX,HON,HPE,HPQ,HRB,HRL,HSIC,HST,HSY,HUM,HWM,IBM,ICE,IDXX,IEX,IFF,ILM

Now we need to create a blank DataFrame and add our data to the data frame one-by-one.

In [7]:
final_dataframe = pd.DataFrame(columns=my_columns)

for symbol_string in symbol_strings:
    batch_api_call_url = f'https://sandbox.iexapis.com/stable/stock/market/batch?symbols={symbol_string}&types=quote&token={IEX_CLOUD_API_TOKEN}'
    #print(symbol_string)
    data = requests.get(batch_api_call_url).json()
    
    #print(data.status_code)
    
    for symbol in symbol_string.split(","):
        final_dataframe = final_dataframe.append(
        pd.Series(
        [
            symbol,
            data[symbol]['quote']['latestPrice'],
            data[symbol]['quote']['peRatio'],
            'N/A'
        ],
        index=my_columns),
        ignore_index=True)
final_dataframe

,Ticker,Price,Price-to-Earnings Ratio,Number of Shares to Buy
0,A,121.98,53.25,N/A
1,AAL,15.80,-1.16,N/A
2,AAP,165.14,24.02,N/A
3,AAPL,137.58,40.89,N/A
4,ABBV,108.21,22.97,N/A
...,...,...,...,...
500,YUM,109.60,33.1,N/A
501,ZBH,155.75,-207.28,N/A
502,ZBRA,391.83,45.06,N/A
503,ZION,43.46,18.7,N/A


## Removing Glamour Stocks

The opposite of a "value stock" is a "glamour stock". 

Since the goal of this strategy is to identify the 50 best value stocks from our universe, our next step is to remove glamour stocks from the DataFrame.

We'll sort the DataFrame by the stocks' price-to-earnings ratio, and drop all stocks outside the top 50.

In [8]:
final_dataframe.sort_values('Price-to-Earnings Ratio', inplace=True)
final_dataframe = final_dataframe[final_dataframe['Price-to-Earnings Ratio'] > 0 ]
final_dataframe = final_dataframe[:50]
final_dataframe.reset_index(inplace = True)
#

In [9]:
final_dataframe.drop('index', axis=1, inplace=True)
final_dataframe

,Ticker,Price,Price-to-Earnings Ratio,Number of Shares to Buy
0,NRG,35.44,2.2,N/A
1,NLOK,21.52,4.05,N/A
2,UNM,22.72,4.79,N/A
3,AIV,5.17,4.97,N/A
4,BIO,601.04,4.98,N/A
5,KIM,14.86,6.7,N/A
6,ALL,109.81,7.78,N/A
7,MET,46.85,7.95,N/A
8,EBAY,51.39,8.06,N/A
9,BIIB,260.01,8.45,N/A


## Calculating the Number of Shares to Buy
We now need to calculate the number of shares we need to buy. 

To do this, we will use the `portfolio_input` function that we created in our momentum project.

I have included this function below.

In [10]:
def portfolio_input():
    global portfolio_size
    portfolio_size = input("Enter the value of your portfolio:")

    try:
        val = float(portfolio_size)
    except ValueError:
        print("That's not a number! \n Try again:")
        portfolio_size = input("Enter the value of your portfolio:")

Use the `portfolio_input` function to accept a `portfolio_size` variable from the user of this script.

In [11]:
portfolio_input()

Enter the value of your portfolio:1000000000


You can now use the global `portfolio_size` variable to calculate the number of shares that our strategy should purchase.

In [12]:
position_size = float(portfolio_size)/len(final_dataframe.index)

for row in final_dataframe.index:
    final_dataframe.loc[row, 'Number of Shares to Buy'] = math.floor(position_size/final_dataframe.loc[row, 'Price'])
    
final_dataframe

,Ticker,Price,Price-to-Earnings Ratio,Number of Shares to Buy
0,NRG,35.44,2.2,564334
1,NLOK,21.52,4.05,929368
2,UNM,22.72,4.79,880281
3,AIV,5.17,4.97,3868471
4,BIO,601.04,4.98,33275
5,KIM,14.86,6.7,1345895
6,ALL,109.81,7.78,182132
7,MET,46.85,7.95,426894
8,EBAY,51.39,8.06,389180
9,BIIB,260.01,8.45,76920


## Building a Better (and More Realistic) Value Strategy
Every valuation metric has certain flaws.

For example, the price-to-earnings ratio doesn't work well with stocks with negative earnings.

Similarly, stocks that buyback their own shares are difficult to value using the price-to-book ratio.

Investors typically use a `composite` basket of valuation metrics to build robust quantitative value strategies. In this section, we will filter for stocks with the lowest percentiles on the following metrics:

* Price-to-earnings ratio
* Price-to-book ratio
* Price-to-sales ratio
* Enterprise Value divided by Earnings Before Interest, Taxes, Depreciation, and Amortization (EV/EBITDA)
* Enterprise Value divided by Gross Profit (EV/GP)

Some of these metrics aren't provided directly by the IEX Cloud API, and must be computed after pulling raw data. We'll start by calculating each data point from scratch.

In [14]:
symbol = 'AAPL'

batch_api_call_url = f'https://sandbox.iexapis.com/stable/stock/market/batch?symbols={symbol}&types=quote,advanced-stats&token={IEX_CLOUD_API_TOKEN}'
data = requests.get(batch_api_call_url).json()
print(data)

# Price-to-earnings ratio
pe_ratio = data[symbol]['quote']['peRatio']

# Price-to-book ratio
pb_ratio = data[symbol]['advanced-stats']['priceToBook']

# Price-to-sales ratio
ps_ratio = data[symbol]['advanced-stats']['priceToSales']

# Enterprise Value divided by Earnings Before Interest, Taxes, Depreciation, and Amortization (EV/EBITDA)
enterprise_value = data[symbol]['advanced-stats']['enterpriseValue']
ebitda = data[symbol]['advanced-stats']['EBITDA']
ev_to_ebitda = enterprise_value/ebitda

# Enterprise Value divided by Gross Profit (EV/GP)
gross_profit = data[symbol]['advanced-stats']['grossProfit']
ev_to_gross_profit = enterprise_value/gross_profit


{'AAPL': {'advanced-stats': {'beta': 1.1608631975162014, 'totalCash': 92848897312, 'currentDebt': 105669158766, 'revenue': 277039069336, 'grossProfit': 108898748270, 'totalRevenue': 282677211491, 'EBITDA': 78656751499, 'revenuePerShare': 16.92, 'revenuePerEmployee': 2068805.05, 'debtToEquity': 4.9608385963371, 'profitMargin': 0.21785877322773445, 'enterpriseValue': 2282116395235, 'enterpriseValueToRevenue': 8.27, 'priceToSales': 8.47, 'priceToBook': 35.77, 'forwardPERatio': 33.60247547306042, 'pegRatio': 3.898002564337028, 'peHigh': 42.39768197975443, 'peLow': 17.13606371994341, 'week52highDate': '2020-08-30', 'week52lowDate': '2020-03-12', 'putCallRatio': 0.3187360376884329, 'companyName': 'Apple Inc', 'marketcap': 2253535763751, 'week52high': 135.61, 'week52low': 57.9, 'week52change': 0.8484429200708611, 'sharesOutstanding': 17181036882, 'float': None, 'avg10Volume': 123135655, 'avg30Volume': 105543629, 'day200MovingAvg': 113.8, 'day50MovingAvg': 128.06, 'employees': 139268, 'ttmEPS'

In [35]:
rv_columns = [
    'Ticker',
    'Price',
    'Number of Shares to Buy',
    'Price-to-Earnings Ratio',
    'PE Percentile',
    'Price-to-Books Ratio',
    'PB Percentile',
    'Price-to-Sales Ratio',
    'PS Percentile',
    'EV/EBITDA',
    'EV/EBITDA Percentile',
    'EV/GP',
    'EV/GP Percentile',
    'RV Score'
]

rv_dataframe = pd.DataFrame(columns=rv_columns)
rv_dataframe

,Ticker,Price,Number of Shares to Buy,Price-to-Earnings Ratio,PE Percentile,Price-to-Books Ratio,PB Percentile,Price-to-Sales Ratio,PS Percentile,EV/EBITDA,EV/EBITDA Percentile,EV/GP,EV/GP Percentile,RV Score


Let's move on to building our DataFrame. You'll notice that I use the abbreviation `rv` often. It stands for `robust value`, which is what we'll call this sophisticated strategy moving forward.

In [36]:
for symbol_string in symbol_strings:
    print(symbol_string)
    batch_api_call_url = f'https://sandbox.iexapis.com/stable/stock/market/batch?symbols={symbol_string}&types=quote,advanced-stats&token={IEX_CLOUD_API_TOKEN}' #f'https://sandbox.iexapis.com/stable/stock/market/batch?symbols={symbol}&types=quote,advanced-stats&token={IEX_CLOUD_API_TOKEN}'
    data = requests.get(batch_api_call_url).json() 
    
    for symbol in symbol_string.split(','):
      
        
        # Price-to-earnings ratio
        pe_ratio = data[symbol]['quote']['peRatio']

        # Price-to-book ratio
        pb_ratio = data[symbol]['advanced-stats']['priceToBook']

        # Price-to-sales ratio
        ps_ratio = data[symbol]['advanced-stats']['priceToSales']

        # Enterprise Value divided by Earnings Before Interest, Taxes, Depreciation, and Amortization (EV/EBITDA)
        enterprise_value = data[symbol]['advanced-stats']['enterpriseValue']
        ebitda = data[symbol]['advanced-stats']['EBITDA']
        try :
            ev_to_ebitda = enterprise_value/ebitda
        except TypeError:
            ev_to_ebitda = np.NaN
            

        # Enterprise Value divided by Gross Profit (EV/GP)
        gross_profit = data[symbol]['advanced-stats']['grossProfit']
        try :
            ev_to_gross_profit = enterprise_value/gross_profit
        except TypeError:
            ev_to_gross_profit = np.NaN
        
        
        rv_dataframe = rv_dataframe.append(
        pd.Series([
            symbol,
            data[symbol]['quote']['latestPrice'],
            'N/A',
            pe_ratio,
            'N/A',
            pb_ratio,
            'N/A',
            ps_ratio,
            'N/A',
            ev_to_ebitda,
            'N/A',
            ev_to_gross_profit,
            'N/A',
            'N/A'
        ], 
        index = rv_columns), 
        ignore_index = True)


A,AAL,AAP,AAPL,ABBV,ABC,ABMD,ABT,ACN,ADBE,ADI,ADM,ADP,ADSK,AEE,AEP,AES,AFL,AIG,AIV,AIZ,AJG,AKAM,ALB,ALGN,ALK,ALL,ALLE,ALXN,AMAT,AMCR,AMD,AME,AMGN,AMP,AMT,AMZN,ANET,ANSS,ANTM,AON,AOS,APA,APD,APH,APTV,ARE,ATO,ATVI,AVB,AVGO,AVY,AWK,AXP,AZO,BA,BAC,BAX,BBY,BDX,BEN,BF.B,BIIB,BIO,BK,BKNG,BKR,BLK,BLL,BMY,BR,BRK.B,BSX,BWA,BXP,C,CAG,CAH,CARR,CAT,CB,CBOE,CBRE,CCI,CCL,CDNS,CDW,CE,CERN,CF,CFG,CHD,CHRW,CHTR,CI,CINF,CL,CLX,CMA,CMCSA
CME,CMG,CMI,CMS,CNC,CNP,COF,COG,COO,COP,COST,COTY,CPB,CPRT,CRM,CSCO,CSX,CTAS,CTL,CTSH,CTVA,CTXS,CVS,CVX,CXO,D,DAL,DD,DE,DFS,DG,DGX,DHI,DHR,DIS,DISCA,DISCK,DISH,DLR,DLTR,DOV,DOW,DPZ,DRE,DRI,DTE,DUK,DVA,DVN,DXC,DXCM,EA,EBAY,ECL,ED,EFX,EIX,EL,EMN,EMR,EOG,EQIX,EQR,ES,ESS,ETFC,ETN,ETR,EVRG,EW,EXC,EXPD,EXPE,EXR,F,FANG,FAST,FB,FBHS,FCX,FDX,FE,FFIV,FIS,FISV,FITB,FLIR,FLS,FLT,FMC,FOX,FOXA,FRC,FRT,FTI,FTNT,FTV,GD,GE,GILD
GIS,GL,GLW,GM,GOOG,GOOGL,GPC,GPN,GPS,GRMN,GS,GWW,HAL,HAS,HBAN,HBI,HCA,HD,HES,HFC,HIG,HII,HLT,HOLX,HON,HPE,HPQ,HRB,HRL,HSIC,HST,HSY,HUM,HWM,IBM,ICE,IDXX,IEX,IFF,ILM

In [13]:
try :
    ev_to_ebitda = enterprise_value/ebitda
except TypeError:
    np.NaN
    

{'week52change': 0.290414, 'week52high': 169.51, 'week52low': 90.41, 'marketcap': 75861250162, 'employees': 10724, 'day200MovingAvg': 134.17, 'day50MovingAvg': 148.64, 'float': 479955020, 'avg10Volume': 1641586.3, 'avg30Volume': 1479579.2, 'ttmEPS': 3.499, 'ttmDividendRate': 0.77, 'companyName': 'Zoetis, Inc.', 'sharesOutstanding': 494883276, 'maxChangePercent': 3.3525, 'year5ChangePercent': 1.69, 'year2ChangePercent': 0.608, 'year1ChangePercent': 0.281583, 'ytdChangePercent': 0.189876, 'month6ChangePercent': 0.10241, 'month3ChangePercent': 0.297048, 'month1ChangePercent': 0.159224, 'day30ChangePercent': 0.165634, 'day5ChangePercent': -0.012817, 'nextDividendDate': None, 'dividendYield': 0.00489706569361443, 'nextEarningsDate': '2020-10-13', 'exDividendDate': '2020-07-01', 'peRatio': 49.1, 'beta': 0.9584403132945124, 'totalCash': 1974321215, 'currentDebt': 549171552, 'revenue': 6518431250, 'grossProfit': 4133660573, 'totalRevenue': 6300683466, 'EBITDA': 2502772553, 'revenuePerShare': 1

## Dealing With Missing Data in Our DataFrame

Our DataFrame contains some missing data because all of the metrics we require are not available through the API we're using. 

You can use pandas' `isnull` method to identify missing data:

In [42]:
rv_dataframe[rv_dataframe.isnull().any(axis=1)]

,Ticker,Price,Number of Shares to Buy,Price-to-Earnings Ratio,PE Percentile,Price-to-Books Ratio,PB Percentile,Price-to-Sales Ratio,PS Percentile,EV/EBITDA,EV/EBITDA Percentile,EV/GP,EV/GP Percentile,RV Score


Dealing with missing data is an important topic in data science.

There are two main approaches:

* Drop missing data from the data set (pandas' `dropna` method is useful here)
* Replace missing data with a new value (pandas' `fillna` method is useful here)

In this tutorial, we will replace missing data with the average non-`NaN` data point from that column. 

Here is the code to do this:

In [38]:
rv_dataframe.columns 

Index(['Ticker', 'Price', 'Number of Shares to Buy', 'Price-to-Earnings Ratio',
       'PE Percentile', 'Price-to-Books Ratio', 'PB Percentile',
       'Price-to-Sales Ratio', 'PS Percentile', 'EV/EBITDA',
       'EV/EBITDA Percentile', 'EV/GP', 'EV/GP Percentile', 'RV Score'],
      dtype='object')

In [41]:
for column in [ 'Price', 'Price-to-Earnings Ratio', 'Price-to-Books Ratio', 'Price-to-Sales Ratio', 'EV/EBITDA', 'EV/GP']:
    print(column)
    rv_dataframe[column].fillna(rv_dataframe[column].mean(), inplace=True)
rv_dataframe

Price
Price-to-Earnings Ratio
Price-to-Books Ratio
Price-to-Sales Ratio
EV/EBITDA
EV/GP


,Ticker,Price,Number of Shares to Buy,Price-to-Earnings Ratio,PE Percentile,Price-to-Books Ratio,PB Percentile,Price-to-Sales Ratio,PS Percentile,EV/EBITDA,EV/EBITDA Percentile,EV/GP,EV/GP Percentile,RV Score
0,A,122.96,N/A,54.73,N/A,7.45,N/A,7.0700,N/A,33.310220,N/A,12.813248,N/A,N/A
1,AAL,16.30,N/A,-1.17,N/A,-1.73,N/A,0.3984,N/A,-5.956781,N/A,1.415938,N/A,N/A
2,AAP,161.81,N/A,23.66,N/A,2.90,N/A,1.1200,N/A,10.881612,N/A,2.471409,N/A,N/A
3,AAPL,132.89,N/A,41.16,N/A,34.68,N/A,8.4600,N/A,29.042397,N/A,22.003266,N/A,N/A
4,ABBV,108.01,N/A,23.15,N/A,12.09,N/A,4.6700,N/A,15.877040,N/A,9.442430,N/A,N/A
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
500,YUM,108.95,N/A,33.50,N/A,-4.00,N/A,6.1000,N/A,24.180838,N/A,10.277744,N/A,N/A
501,ZBH,153.15,N/A,-209.24,N/A,2.71,N/A,4.4100,N/A,26.179409,N/A,7.726793,N/A,N/A
502,ZBRA,392.63,N/A,44.89,N/A,10.64,N/A,4.6800,N/A,29.768834,N/A,10.767936,N/A,N/A
503,ZION,44.11,N/A,18.86,N/A,1.00,N/A,2.4400,N/A,10.474847,N/A,2.700809,N/A,N/A


Now, if we run the statement from earlier to print rows that contain missing data, nothing should be returned:

In [43]:
rv_dataframe[rv_dataframe.isnull().any(axis=1)]

,Ticker,Price,Number of Shares to Buy,Price-to-Earnings Ratio,PE Percentile,Price-to-Books Ratio,PB Percentile,Price-to-Sales Ratio,PS Percentile,EV/EBITDA,EV/EBITDA Percentile,EV/GP,EV/GP Percentile,RV Score


## Calculating Value Percentiles

We now need to calculate value score percentiles for every stock in the universe. More specifically, we need to calculate percentile scores for the following metrics for every stock:

* Price-to-earnings ratio
* Price-to-book ratio
* Price-to-sales ratio
* EV/EBITDA
* EV/GP

Here's how we'll do this:

In [44]:
metrics  = {
    'Price-to-Earnings Ratio': 'PE Percentile',
    'Price-to-Books Ratio':'PB Percentile',
    'Price-to-Sales Ratio': 'PS Percentile',
    'EV/EBITDA': 'EV/EBITDA Percentile',
    'EV/GP': 'EV/GP Percentile'
}

for metric in metrics.keys():
    for row in rv_dataframe.index:
        rv_dataframe.loc[row, metrics[metric]] = stats.percentileofscore(rv_dataframe[metric], rv_dataframe.loc[row, metric])
        
        
rv_dataframe

,Ticker,Price,Number of Shares to Buy,Price-to-Earnings Ratio,PE Percentile,Price-to-Books Ratio,PB Percentile,Price-to-Sales Ratio,PS Percentile,EV/EBITDA,EV/EBITDA Percentile,EV/GP,EV/GP Percentile,RV Score
0,A,122.96,N/A,54.73,85.9406,7.45,72.0792,7.0700,80.6931,33.310220,84.5545,12.813248,78.0198,N/A
1,AAL,16.30,N/A,-1.17,14.0594,-1.73,4.55446,0.3984,2.9703,-5.956781,2.77228,1.415938,5.94059,N/A
2,AAP,161.81,N/A,23.66,45.8416,2.90,40.396,1.1200,16.6337,10.881612,26.1386,2.471409,12.4752,N/A
3,AAPL,132.89,N/A,41.16,75.4455,34.68,96.0396,8.4600,85.1485,29.042397,80.9901,22.003266,95.0495,N/A
4,ABBV,108.01,N/A,23.15,45.1485,12.09,84.5545,4.6700,67.3267,15.877040,48.5149,9.442430,64.7525,N/A
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
500,YUM,108.95,N/A,33.50,66.3366,-4.00,4.15842,6.1000,76.2376,24.180838,74.4554,10.277744,68.7129,N/A
501,ZBH,153.15,N/A,-209.24,1.9802,2.71,38.6139,4.4100,61.7822,26.179409,77.8218,7.726793,50.099,N/A
502,ZBRA,392.63,N/A,44.89,78.0198,10.64,82.7723,4.6800,67.5248,29.768834,82.1782,10.767936,70.099,N/A
503,ZION,44.11,N/A,18.86,37.2277,1.00,10.099,2.4400,40.7921,10.474847,23.9604,2.700809,14.2574,N/A


## Calculating the RV Score
We'll now calculate our RV Score (which stands for Robust Value), which is the value score that we'll use to filter for stocks in this investing strategy.

The RV Score will be the arithmetic mean of the 4 percentile scores that we calculated in the last section.

To calculate arithmetic mean, we will use the mean function from Python's built-in statistics module.

In [45]:
from statistics import mean 

for row in rv_dataframe.index:
    value_percentiles = [] 
    for metric in metrics.keys():
        value_percentiles.append(rv_dataframe.loc[row, metrics[metric]])
        
    rv_dataframe.loc[row, "RV Score"] = mean(value_percentiles)
    #print(value_percentiles)
rv_dataframe

,Ticker,Price,Number of Shares to Buy,Price-to-Earnings Ratio,PE Percentile,Price-to-Books Ratio,PB Percentile,Price-to-Sales Ratio,PS Percentile,EV/EBITDA,EV/EBITDA Percentile,EV/GP,EV/GP Percentile,RV Score
0,A,122.96,N/A,54.73,85.9406,7.45,72.0792,7.0700,80.6931,33.310220,84.5545,12.813248,78.0198,80.2574
1,AAL,16.30,N/A,-1.17,14.0594,-1.73,4.55446,0.3984,2.9703,-5.956781,2.77228,1.415938,5.94059,6.05941
2,AAP,161.81,N/A,23.66,45.8416,2.90,40.396,1.1200,16.6337,10.881612,26.1386,2.471409,12.4752,28.297
3,AAPL,132.89,N/A,41.16,75.4455,34.68,96.0396,8.4600,85.1485,29.042397,80.9901,22.003266,95.0495,86.5347
4,ABBV,108.01,N/A,23.15,45.1485,12.09,84.5545,4.6700,67.3267,15.877040,48.5149,9.442430,64.7525,62.0594
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
500,YUM,108.95,N/A,33.50,66.3366,-4.00,4.15842,6.1000,76.2376,24.180838,74.4554,10.277744,68.7129,57.9802
501,ZBH,153.15,N/A,-209.24,1.9802,2.71,38.6139,4.4100,61.7822,26.179409,77.8218,7.726793,50.099,46.0594
502,ZBRA,392.63,N/A,44.89,78.0198,10.64,82.7723,4.6800,67.5248,29.768834,82.1782,10.767936,70.099,76.1188
503,ZION,44.11,N/A,18.86,37.2277,1.00,10.099,2.4400,40.7921,10.474847,23.9604,2.700809,14.2574,25.2673


## Selecting the 50 Best Value Stocks¶

As before, we can identify the 50 best value stocks in our universe by sorting the DataFrame on the RV Score column and dropping all but the top 50 entries.

In [46]:
rv_dataframe.sort_values('RV Score', ascending=False, inplace=True)
rv_dataframe = rv_dataframe[:50]
rv_dataframe.reset_index(drop=True, inplace=True)
rv_dataframe

,Ticker,Price,Number of Shares to Buy,Price-to-Earnings Ratio,PE Percentile,Price-to-Books Ratio,PB Percentile,Price-to-Sales Ratio,PS Percentile,EV/EBITDA,EV/EBITDA Percentile,EV/GP,EV/GP Percentile,RV Score
0,NOW,557.04,N/A,160.21,98.4158,45.60,96.8317,28.61,99.604,224.279385,99.802,33.667025,99.604,98.8515
1,PAYC,473.88,N/A,156.95,97.8218,47.56,97.2277,34.48,100,97.713103,98.6139,38.304049,100,98.7327
2,ADSK,312.55,N/A,159.30,98.2178,967.29,100,18.58,97.6238,104.116535,99.2079,20.032559,93.6634,97.7426
3,DXCM,365.86,N/A,151.55,97.6238,23.32,92.8713,19.16,97.8218,93.611218,98.4158,27.080208,97.8218,96.9109
4,IDXX,502.06,N/A,98.04,95.0495,157.24,98.8119,17.92,97.0297,65.448616,92.6733,30.674955,98.8119,96.4752
5,AMD,95.06,N/A,127.47,96.6337,29.17,95.2475,13.32,94.4554,76.043445,97.4257,28.649327,98.2178,96.396
6,NVDA,525.20,N/A,85.42,93.0693,21.94,92.0792,21.82,99.0099,66.548848,96.4356,35.238579,99.802,96.0792
7,MKTX,582.36,N/A,79.64,92.0792,24.38,93.4653,34.46,99.802,57.014942,91.4851,32.444240,99.4059,95.2475
8,ISRG,838.00,N/A,94.08,94.2574,10.62,82.3762,22.31,99.4059,68.668323,96.8317,30.985154,99.0099,94.3762
9,TYL,450.54,N/A,95.49,94.6535,9.66,80.7921,16.12,96.4356,88.556627,98.2178,31.779740,99.2079,93.8614


## Calculating the Number of Shares to Buy
We'll use the `portfolio_input` function that we created earlier to accept our portfolio size. Then we will use similar logic in a for loop to calculate the number of shares to buy for each stock in our investment universe.

In [52]:
portfolio_input()

Enter the value of your portfolio:2500000


In [53]:
position_size = float(portfolio_size)/len(final_dataframe.index)

for row in rv_dataframe.index:
    rv_dataframe.loc[row, 'Number of Shares to Buy'] = math.floor(position_size/rv_dataframe.loc[row, 'Price'])
    
rv_dataframe

c:\users\i2989\appdata\local\programs\python\python38\lib\site-packages\pandas\core\indexing.py:1765: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  isetter(loc, value)


,Ticker,Price,Number of Shares to Buy,Price-to-Earnings Ratio,PE Percentile,Price-to-Books Ratio,PB Percentile,Price-to-Sales Ratio,PS Percentile,EV/EBITDA,EV/EBITDA Percentile,EV/GP,EV/GP Percentile,RV Score
0,NOW,557.04,89,160.21,98.4158,45.60,96.8317,28.61,99.604,224.279385,99.802,33.667025,99.604,98.8515
1,PAYC,473.88,105,156.95,97.8218,47.56,97.2277,34.48,100,97.713103,98.6139,38.304049,100,98.7327
2,ADSK,312.55,159,159.30,98.2178,967.29,100,18.58,97.6238,104.116535,99.2079,20.032559,93.6634,97.7426
3,DXCM,365.86,136,151.55,97.6238,23.32,92.8713,19.16,97.8218,93.611218,98.4158,27.080208,97.8218,96.9109
4,IDXX,502.06,99,98.04,95.0495,157.24,98.8119,17.92,97.0297,65.448616,92.6733,30.674955,98.8119,96.4752
5,AMD,95.06,525,127.47,96.6337,29.17,95.2475,13.32,94.4554,76.043445,97.4257,28.649327,98.2178,96.396
6,NVDA,525.20,95,85.42,93.0693,21.94,92.0792,21.82,99.0099,66.548848,96.4356,35.238579,99.802,96.0792
7,MKTX,582.36,85,79.64,92.0792,24.38,93.4653,34.46,99.802,57.014942,91.4851,32.444240,99.4059,95.2475
8,ISRG,838.00,59,94.08,94.2574,10.62,82.3762,22.31,99.4059,68.668323,96.8317,30.985154,99.0099,94.3762
9,TYL,450.54,110,95.49,94.6535,9.66,80.7921,16.12,96.4356,88.556627,98.2178,31.779740,99.2079,93.8614


## Formatting Our Excel Output

We will be using the XlsxWriter library for Python to create nicely-formatted Excel files.

XlsxWriter is an excellent package and offers tons of customization. However, the tradeoff for this is that the library can seem very complicated to new users. Accordingly, this section will be fairly long because I want to do a good job of explaining how XlsxWriter works.

In [54]:
writer = pd.ExcelWriter('value_strategy.xlsx', engine='xlsxwriter')
rv_dataframe.to_excel(writer, sheet_name='Value Strategy', index = False)

## Creating the Formats We'll Need For Our .xlsx File
You'll recall from our first project that formats include colors, fonts, and also symbols like % and $. We'll need four main formats for our Excel document:

* String format for tickers
* \$XX.XX format for stock prices
* \$XX,XXX format for market capitalization
* Integer format for the number of shares to purchase
* Float formats with 1 decimal for each valuation metric

Since we already built some formats in past sections of this course, I've included them below for you. Run this code cell before proceeding.

In [55]:
background_color = '#0a0a23'
font_color = '#ffffff'

string_template = writer.book.add_format(
        {
            'font_color': font_color,
            'bg_color': background_color,
            'border': 1
        }
    )

dollar_template = writer.book.add_format(
        {
            'num_format':'$0.00',
            'font_color': font_color,
            'bg_color': background_color,
            'border': 1
        }
    )

integer_template = writer.book.add_format(
        {
            'num_format':'0',
            'font_color': font_color,
            'bg_color': background_color,
            'border': 1
        }
    )

float_template = writer.book.add_format(
        {
            'num_format':'0',
            'font_color': font_color,
            'bg_color': background_color,
            'border': 1
        }
    )

percent_template = writer.book.add_format(
        {
            'num_format':'0.0%',
            'font_color': font_color,
            'bg_color': background_color,
            'border': 1
        }
    )

In [56]:
column_formats = {
                    'A': ['Ticker', string_template],
                    'B': ['Price', dollar_template],
                    'C': ['Number of Shares to Buy', integer_template],
                    'D': ['Price-to-Earnings Ratio', float_template],
                    'E': ['PE Percentile', percent_template],
                    'F': ['Price-to-Book Ratio', float_template],
                    'G': ['PB Percentile',percent_template],
                    'H': ['Price-to-Sales Ratio', float_template],
                    'I': ['PS Percentile', percent_template],
                    'J': ['EV/EBITDA', float_template],
                    'K': ['EV/EBITDA Percentile', percent_template],
                    'L': ['EV/GP', float_template],
                    'M': ['EV/GP Percentile', percent_template],
                    'N': ['RV Score', percent_template]
                 }

for column in column_formats.keys():
    writer.sheets['Value Strategy'].set_column(f'{column}:{column}', 25, column_formats[column][1])
    writer.sheets['Value Strategy'].write(f'{column}1', column_formats[column][0], column_formats[column][1])

## Saving Our Excel Output
As before, saving our Excel output is very easy:

In [57]:
writer.save()